In [1]:
%pwd

'/Users/ryandevera/data-science/umn_environments/Constrained-Deep-Learning-Survey/notebooks'

In [2]:
%cd ..

/Users/ryandevera/data-science/umn_environments/Constrained-Deep-Learning-Survey


/Users/ryandevera/.virtualenvs/cdl-survey/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
# stdlib
import math
import random

# third party
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow.compat.v1 as tf
import tensorflow_constrained_optimization as tfco
import warnings

# first party
from cdlsurvey.data import get_data
from cdlsurvey.metrics import get_exp_error_rate_constraints
from cdlsurvey.models import Model
from cdlsurvey.utils import training_helper

# Disable eager execution
tf.disable_eager_execution()

# suppress warnings
warnings.filterwarnings('ignore')

# For plotting in notebook
%matplotlib inline

2024-01-23 11:52:30.575163: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Reading and processing dataset

We load the [UCI Adult Dataset](https://archive.ics.uci.edu/dataset/2/adult) and we do some pre-processing. The way this was done in the `TFCO` tutorial is by:

-  Binarizing the categorial variables
-  Binning and binarizing the continuous variables.

The dataset is based on census data and the goal is to predict whether someone's income is over 50k. We construct four protected groups, two based on gender (Male and Female) and two based on race (White and Black). The pre-processing of the features based on the works [Fairness Constraints: Mechanisms for Fair Classification](https://arxiv.org/pdf/1507.05259.pdf) and [Satisfying Real-World Goals with Dataset Constraints](https://arxiv.org/pdf/1606.07558.pdf).

The fairness goal is that of equal opportunity. That is, we would like the **true positive rates** of our classifier on the protected groups to match that of the overall dataset.

In [4]:
CATEGORICAL_COLUMNS = [
    'workclass', 'education', 'marital_status', 'occupation', 'relationship',
    'race', 'gender', 'native_country'
]
CONTINUOUS_COLUMNS = [
    'age', 'capital_gain', 'capital_loss', 'hours_per_week', 'education_num'
]
COLUMNS = [
    'age', 'workclass', 'fnlwgt', 'education', 'education_num',
    'marital_status', 'occupation', 'relationship', 'race', 'gender',
    'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
    'income_bracket'
]
LABEL_COLUMN = 'label'

PROTECTED_COLUMNS = [
    'gender_Female', 'gender_Male', 'race_White', 'race_Black'
]

# Data Function & Gathering

The cells below will load in the Adult dataset and featureize the data.

In [5]:
# Gather the data
train_df, test_df, FEATURE_NAMES = get_data()

In [6]:
# Make sure there are positive labels
train_df['label'].sum(), test_df['label'].sum()

(7841, 3846)

# Model

We will use a simple 2-layer MLP and predict positively or negatively based on threshold at 0.

The model code can be found here [Fairness MLP Model - Sun Group](https://github.com/sun-umn/Constrained-Deep-Learning-Survey/blob/main/cdlsurvey/models.py). In the code, we initialize the placeholders and model. In the `build_train_op`, we set up the constrained optimization problem. We create a rate context for the entire dataset, and compue the overall false positive rate as teh positive prediction rate on the negatively labeled subset.

We then can construct a constraint for each of the protected groups based on the difference between the true positive rates of the protected groups and that of the overall dataset. We then construct a minimization problem using `RateMinimizationProblem` and use the `ProxyLagrangianOptimizerV1` as the solver. `build_train_op` initializes a training operation which will later be used to actually train the model.

# Baseline without constraints

We now declare the model, build the training op, and then perform the training. We use a 2-layer MLP, and train using the Adam optimizer with learning rate 0.01, with minibatch size of 100 over 40 epochs. We train without fairness constraints to show the baseline performance. We see that without training for fairness, we obtain a high fairness evaluation.

In [7]:
model = Model(
    tpr_max_diff=0.05,
    protected_columns=PROTECTED_COLUMNS,
    feature_names=FEATURE_NAMES,
    label_column=LABEL_COLUMN,
)
model.build_train_op(0.01, unconstrained=True)

# training_helper returns the list of errors and violations over each epoch.
train_errors, train_violations, test_errors, test_violations = training_helper(
    model,
    train_df,
    test_df,
    100,
    num_iterations_per_loop=326,
    num_loops=40,
)

In [8]:
print("Train Error", train_errors[-1])
print("Train Violation", max(train_violations[-1]))
print()
print("Test Error", test_errors[-1])
print("Test Violation", max(test_violations[-1]))

Train Error 0.12118792420380209
Train Violation -0.04409253750329134

Test Error 0.14532276887169093
Test Violation 0.02092328385872866


# Training with fairness constraint

We now show train with the constraints using the procedure of [Two-Player Games for Efficient Non=convex Constrained Optimization](https://arxiv.org/abs/1804.06500) and returning the last solution found. We see that the fairness violation improves.

We allow an additive fairness slack of 0.05. That is, when training and evaluating the fairness constraints, the true postive rate difference between protected groups has to be at least that of the overall dataset up to a slack of at most 0.05. Thus, the fairness constraints woul dbe of the form $TPR_p \geq TPR - 0.05$, where $TPR_p$ and TPR denotes the true postive rates of the protected group and the overall dataset, respectively.

In [9]:
model = Model(
    tpr_max_diff=0.05,
    protected_columns=PROTECTED_COLUMNS,
    feature_names=FEATURE_NAMES,
    label_column=LABEL_COLUMN,
)
model.build_train_op(0.01, unconstrained=False)

# training_helper returns the list of errors and violations over each epoch.
train_errors, train_violations, test_errors, test_violations = training_helper(
      model,
      train_df,
      test_df,
      100,
      num_iterations_per_loop=326,
      num_loops=40)

In [10]:
print("Train Error", train_errors[-1])
print("Train Violation", max(train_violations[-1]))
print()
print("Test Error", test_errors[-1])
print("Test Violation", max(test_violations[-1]))

Train Error 0.12192500230336906
Train Violation -0.010769920490356633

Test Error 0.14581413918064
Test Violation 0.031323699875369315


In [11]:
print("Train Error", np.mean(train_errors))
print("Train Violation", max(np.mean(train_violations, axis=0)))
print()
print("Test Error", np.mean(test_errors))
print("Test Violation", max(np.mean(test_violations, axis=0)))

Train Error 0.13148859064525045
Train Violation -0.012193528525327266

Test Error 0.1450049137030895
Test Violation 0.017228812057510222


# Improve using best iterate instead of last iterate

As discussed in [Optimization with Non-differentiable Constraints with Applications to Fairness, Recall, Churn, and Other Goals](https://arxiv.org/abs/1809.04198), the last iterate may not be the best choice and suggests a simple heuristic to choose the best iterate out of the ones found after each epoch. The heuristic proceeds by ranking each of the solutions based on accuracy and fairness separately with respect to the training data. Any solutions which satisfy the constraints are equally ranked top in terms of fairness. Each solution thus has two ranks. Then, the solution chosen is the one with the smallest maximum of the two ranks. We see that this improves the fairness and can find a better accuracy / fairness trade-off on the training data.

The solution can be calculated using `find_best_candidate_index` given the list of training errors and vioilations associated with each of the epochs.

In [12]:
best_cand_index = tfco.find_best_candidate_index(train_errors, train_violations)

print("Train Error", train_errors[best_cand_index])
print("Train Violation", max(train_violations[best_cand_index]))
print()
print("Test Error", test_errors[best_cand_index])
print("Test Violation", max(test_violations[best_cand_index]))

Train Error 0.12192500230336906
Train Violation -0.010769920490356633

Test Error 0.14581413918064
Test Violation 0.031323699875369315


# Using stochastic solutions

As discussed in [Two-Player Games for Efficient Non=convex Constrained Optimization](https://arxiv.org/abs/1804.06500), neither the best nor last iterate will come with theoretical guarantees. One can instead use randomlized solutions, which come with theoretical gurantees. However, as discussed in [Optimization with Non-differentiable Constraints with Applications to Fairness, Recall, Churn, and Other Goals](https://arxiv.org/abs/1809.04198), there may not always be a clear practical benefit. We show how to use these solutions here for sake of completeness.

## T-stochastic solution

The first and simplest randomized solution suggested is the T-stochastic, which simply takes the average of all the iterates found at each epoch.

In [13]:
print("Train Error", np.mean(train_errors))
print("Train Violation", max(np.mean(train_violations, axis=0)))
print()
print("Test Error", np.mean(test_errors))
print("Test Violation", max(np.mean(test_violations, axis=0)))

Train Error 0.13148859064525045
Train Violation -0.012193528525327266

Test Error 0.1450049137030895
Test Violation 0.017228812057510222


## M-stochastic solution

[Two-Player Games for Efficient Non=convex Constrained Optimization](https://arxiv.org/abs/1804.06500) presents a method which shrinks down the **T-stochastic** solution down to the one that is supported on at most (m + 1) points where m is the number of constraints and is guaranteed to be at least as good as the T-stochastic solution. Here we see that indeed there is benefit in performing the shrinking.

This solution can be computed using the `find_best_candidate_distribution` by passing in the training errors and violations found at each epoch and return the weight of each constituent. We see that indeed, it is sparse.

In [14]:
cand_dist = tfco.find_best_candidate_distribution(train_errors, train_violations)
print(cand_dist)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


In [15]:
m_stoch_error_train, m_stoch_violations_train = get_exp_error_rate_constraints(cand_dist, train_errors, train_violations)
m_stoch_error_test, m_stoch_violations_test = get_exp_error_rate_constraints(cand_dist, test_errors, test_violations)

print("Train Error", m_stoch_error_train)
print("Train Violation", max(m_stoch_violations_train))
print()
print("Test Error", m_stoch_error_test)
print("Test Violation", max(m_stoch_violations_test))

Train Error 0.12192500230336906
Train Violation -0.010769920490356633

Test Error 0.14581413918064
Test Violation 0.031323699875369315
